In [ ]:
import requests
import json
from bs4 import BeautifulSoup
import urllib.request
import base64
import os
from tika import parser
import PyPDF2

### parameter
- seviceKey : id
- target : admrul
- query : 검색어(default: * \)
- numOfRows : 한 페이지 결과수
- pageNo : 페이지 번호

In [ ]:
ServiceKey ='qHBjb9yik8lx%2B3bJ3tLeWux4VlZOlRcSe0otBb3Yh4T5c24Yia5vI6e%2BVFFUYHRwWl8FgTAvgRJjWf9XiIlpQg%3D%3D'

In [ ]:
url = 'http://apis.data.go.kr/1170000/law/admrulSearchList.do?'
url += 'serviceKey=' + ServiceKey + '&query=*&target=admrul&numOfRows=10&pageNo=5'

In [ ]:
res = requests.get(url).text

In [ ]:
soup = BeautifulSoup(res,'html.parser')

In [ ]:
crawl_ls = soup.select('admrul')
id_ls = []
for crawl in crawl_ls:
    crawl = str(crawl)
    
    start = crawl.index('ID=')
    crawl = crawl[start+3:]
    end = crawl.index('&')
    crawl = crawl[:end]

    id_ls.append(crawl)

In [ ]:
params = {}
url_detail = 'http://www.law.go.kr/LSW/admRulAttFlList.do' 
file_ls = []
for admid in id_ls:
    file_info = []
    params.update({'admRulSeq':admid})
    res = requests.post(url_detail, data=params).text
    soup = BeautifulSoup(res,'html.parser')
    files = soup.select('ul li a')
    for file in files:
        file_info.append([file['href'],file.text])
    file_ls.append(file_info)

In [ ]:
def download(url, filename):
    if os.path.exists('out.pdf') is False:
        with open('out.pdf',"wb") as file:
            res = requests.get(url)
            file.write(res.content)
    
#     pdf = PyPDF2.PdfFileReader(open('out.pdf','rb'))
#     page = pdf.getNumPages()
#     with open('a.txt','wb') as output:
#         for i in range(page):
#             page = pdf.getPage(i)
#             print(page.extractText())
#             output.write(page.extractText().encode('utf-8'))
#             if i == 10:
#                 break
    with open("a.txt","w",encoding='utf-8') as fileOut:
        parsed = parser.from_file('out.pdf')
        fileOut.write(parsed['content'])

In [ ]:
url_file = 'http://www.law.go.kr/LSW/'
for files in file_ls:
    for file in files:
        if 'pdf' in file[1]:
            print(url_file+file[0])
            print(file[1])
            download(url_file+file[0],file[1])
            
#         de_buf = base64.b64decode(buf).decode('utf-8')
#         print(base64.b64encode(de_buf.encode('utf-8')))
#         res = requests.get(url_file+file[0], stream=True)
#         with open('~/Downloads/'+file[1], 'r') as f:
#           f.read() 
    print('\n')